### Description

Yandex.Algorithm ML

Каждый из файлов субтитров в датасете OpenSubtitles [2], который мы использовали в качестве источника реплик и разговоров, содержит упорядоченный набор реплик. В большинстве случаев, каждая реплика – это ответ на предыдущую, в разговоре между двумя персонажами фильма. Мы случайно выбрали эпизоды этих разговоров в качестве наших тренировочных и тестовых примеров.

Каждый эпизод состоит из двух частей – контекста (Context) и финальной реплики (Reply). Например,

- context_2: Персонаж A говорит реплику 

- context_1: Персонаж B отвечает на нее 

- context_0: Персонаж А произносит вторую реплику 

reply: Персонаж B отвечает на вторую реплику 
Контекстная часть может состоять из трех реплик (как в примере) – в 50% случаев, двух – в 25%, и одного – в оставшихся 25% случаев. Финальная реплика (Reply) всегда завершает любой эпизод, то есть следует за контекстом (Context). Задача участников – найти наиболее подходящую и интересную реплику для данного контекста среди предложенных кандидатов (числом до 6), случайно выбранных из топа кандидатов, возвращенных бейзлайном высокого качества, натренированным командой Алисы (который, в свою очередь, отобрал кандидатов среди всех возможных реплик OpenSubtitles).

Все реплики-кандидаты размечены асессорами на сервисе Яндекс.Толока с помощью следующей инструкции для разметки:

- Good (2): реплика уместна (имеет смысл для данного контекста) и интересна (нетривиальна, специфична именно для данного контекста, мотивирует продолжать разговор)

- Neutral (1): реплика уместна (имеет смысл для данного контекста), но не интересна (тривиальна, не специфична для данного контекста и скорее подталкивает пользователя закончить разговор)

- Bad (0): реплика не имеет никакого смысла в данном контексте

Каждая метка в тренировочной части датасета (и только в ней), сопровождается также уверенностью (confidence) – числом в интервале от 0 до 1 – которое показывает насколько уверенными в своей разметке были асессоры с Толоки, совместно предложившие данную метку. Мы хотим обратить особое внимание участников на эту информацию, она может быть очень полезна при обучении их моделей.

Мы хотим особо отметить, что все участники имеют право скачать датасет OpenSubtitles [2], который использовался для подготовки датасета и применять его для тренировки своих моделей по своему усмотрению.

In [1]:
import pandas as pd, numpy as np

In [ ]:
!ls data

- context_id – идентификатор эпизода
- context_2,context_1,context_0 – текст реплик, предшествующих финальной (может состоять из трех частей)
- reply_id – идентификатор реплики-кандидата
- reply – текст реплики-кандидата
- label – метка реплики-кандидата (good, neutral или bad)
- confidence - уверенность в метке реплики-кандидата (число от 0 до 1)

### Load

In [2]:
df = pd.read_csv('data/train.tsv', sep='\t', quotechar=' ', header = None)
df.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply', 'label', 'confidence']
test = pd.read_csv('data/public.tsv', sep='\t', quotechar = ' ', header = None)
test.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply']

In [ ]:
df.head(6)

In [ ]:
test.head(8)

y - label, and prob

### Prep

#### Label

In [3]:
def label_enc(x ,reverse = False):
    if reverse == False:
        if x == 'bad':
            return 0
        elif x == 'neutral':
            return 1
        else:
            return 2
    else:
        if x == 0:
            return 'bad'
        elif x == 1:
            return 'neutral'
        else:
            return 'good'

In [4]:
df['label'] = df['label'].apply(lambda x: label_enc(x))

#### Scorer

In [5]:
from sklearn.metrics import make_scorer

In [6]:
def DCG(label): return sum([float(label[i]/np.log2(i+2)) for i in range(len(label))])

def nDCG(label, best_label):
    label, best_label = DCG(label), DCG(best_label)
    if label != 0 and best_label != 0:
        return label/best_label
    else:
        return 0

scorer = make_scorer(nDCG)

#### Nan clearing
Wanted more clever way to dell and fill nan, but dropna or fillna, will work good.

In [7]:
df.fillna('-', inplace=True)
test.fillna('-', inplace=True)

#### Vectorizer

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

import scipy.sparse as sps

In [9]:
def Vect(df, test):
    tfidf = TfidfVectorizer()
    
    context_0 = tfidf.fit_transform(df['context_0'])
    context_0_t = tfidf.transform(test['context_0'])
    
    context_1 = tfidf.fit_transform(df['context_1'])
    context_1_t = tfidf.transform(test['context_1'])

    
    context_2 = tfidf.fit_transform(df['context_2'])
    context_2_t = tfidf.transform(test['context_1'])

    reply = tfidf.fit_transform(df['reply'])
    reply_t = tfidf.transform(test['reply'])
    
    return sps.hstack((context_0, context_1, context_2, reply)), \
           sps.hstack((context_0_t, context_1_t, context_2_t, reply_t))



X_train, X_test = Vect(df, test)
y = df['label'].values

### CV

In [10]:
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression

import datetime

In [ ]:
def Testing_grid_reg(X_train, Y_train, c=[10.0 ** i for i in range(-9, 8)]):
    grid = {'C': c,
           'penalty' : ['l1', 'l2']}
    
    clf = LogisticRegression(n_jobs=1)
    
    start_time = datetime.datetime.now()
    gs = GridSearchCV(clf, scoring=scorer, param_grid=grid, cv=5, 
                      return_train_score=True, n_jobs=-1, verbose = True)
    gs.fit(X_train,Y_train)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']
    
    for mean, std, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    
    return max(means), gs.best_params_

In [ ]:
Testing_grid_reg(X, y)

### Predict

In [11]:
clf = LogisticRegression().fit(X_train, y)

In [12]:
pred = clf.predict(X_test)
pred_proba = clf.predict_proba(X_test)

In [13]:
pred_proba  = [i.max() for i in pred_proba]

#### Save subm

In [14]:
test['confidence'] = pred_proba
test['label']  = pred

In [28]:
test.sort_values(by=['context_id', 'confidence'], ascending=False)[['context_id', 'reply_id']].to_csv('subm.csv', encoding='utf-8', sep=' ', index=False)